In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import numpy as np

Функция, которая вырезает html синтаксис из строки

In [ ]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

Получает число страниц для данного показателя на сайте banki.ru

In [ ]:
def get_max_page(page):
    max_page = html_stripper(page.find('span',attrs={'data-bind':'total-items'}))
    if max_page=='None':
        return 0
    else:
        return int((int(max_page)-1)/50)+1

Получает номера лицензий со значениями со страницы

In [ ]:
def get_values(page):
    licens = page.findAll('div',attrs={'class':'font-size-small color-gray-burn'})
    licens = html_stripper(licens)
    licens = re.split('№ |,',licens)
    values = page.findAll('td',attrs={'class':'text-align-right'})
    zap = {}
    for i in range(int(len(licens)/3)):
        zap.update({licens[1+i*3]:int(re.sub('−','-',re.sub(' |\n|,','',html_stripper(values[4*i]))))})
    return zap

Массив доступных дат

In [ ]:
dates = []
for year in range(2008,2018):
    for month in range(1,13):
        if month>=10:
            dates.append(str(year)+'-'+str(month)+'-01')
        else:
            dates.append(str(year)+'-0'+str(month)+'-01')
dates = dates[2:-9]

Массив названий фичей и их номеров для ссылки (предварительно сохранил код всплывающего окна)

In [ ]:
file = open('features', 'r')
page = file.read()
file.close()
page = BeautifulSoup(page, 'lxml')
features = page.findAll('a',attrs={'class':'rating-parameter-list--item__link active'})
names = html_stripper(features)
IDS = re.split('PROPERTY_ID=|">',str(features))
names = re.split('                                                    |                                                |                                            ',names)
d = {}
for i in range(1,173,2):
    d.update({IDS[i]:names[i]})

Удаляем наименее информативные показатели и дублирующиеся показатели

In [ ]:
d.pop('10')
d.pop('210')
d.pop('220')
d.pop('230')
d.pop('240')
d.pop('310')
d.pop('320')
d.pop('330')
d.pop('340')
d.pop('420')
d.pop('421')
d.pop('430')
d.pop('431')
d.pop('440')
d.pop('441')
d.pop('450')
d.pop('451')
d.pop('460')
d.pop('461')
d.pop('520')
d.pop('521')
d.pop('530')
d.pop('531')
d.pop('540')
d.pop('541')
d.pop('550')
d.pop('551')
d.pop('560')
d.pop('561')
d.pop('20')
d.pop('25')
d.pop('30')

Для каждого месяца создаем таблицу со всеми показателями всех банков в этот месяц

In [ ]:
for date in ['2016-04-01']:
    print(date)
    
    stat_id='100'
    url = 'http://www.banki.ru/banks/ratings/?SEARCH_NAME=&SEARCH_REGN=&search%5Btype%5D=name&sort_param=rating&sort_order=ASC&PROPERTY_ID='
    url += stat_id + '&REGION_ID=0&date1='
    url += date + '&date2=2016-12-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0#search_label'
    page = requests.get(url)
    page = page.content
    page = BeautifulSoup(page, 'lxml')
    max_page = get_max_page(page)

    tabl = {}
    tabl.update(get_values(page))

    for i in range(2,max_page+1):
        url = 'http://www.banki.ru/banks/ratings/?PAGEN_1=' + str(i)
        url += '&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID='
        url += stat_id + '&REGION_ID=0&date1='
        url += date + '&date2=2016-12-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0#search_label'
        page = requests.get(url)
        page = page.content
        page = BeautifulSoup(page, 'lxml')
        tabl.update(get_values(page))

    base = pd.DataFrame(tabl,index=[int(stat_id)])

    for stat_id in list(d.keys()):
        
        if int(stat_id)%100==0:
            print(stat_id)
        
        if stat_id=='100':
            continue

        else:
            url = 'http://www.banki.ru/banks/ratings/?SEARCH_NAME=&SEARCH_REGN=&search%5Btype%5D=name&sort_param=rating&sort_order=ASC&PROPERTY_ID='
            url += stat_id + '&REGION_ID=0&date1='
            url += date + '&date2=2016-12-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0#search_label'
            page = requests.get(url)
            page = page.content
            page = BeautifulSoup(page, 'lxml')
            max_page = get_max_page(page)

            tabl = {}
            tabl.update(get_values(page))

            for i in range(2,max_page+1):
                url = 'http://www.banki.ru/banks/ratings/?PAGEN_1=' + str(i)
                url += '&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID='
                url += stat_id + '&REGION_ID=0&date1='
                url += date + '&date2=2016-12-01&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0#search_label'
                page = requests.get(url)
                page = page.content
                page = BeautifulSoup(page, 'lxml')
                tabl.update(get_values(page))

            base = base.append(pd.DataFrame(tabl,index=[int(stat_id)]))

    base.to_csv(date+'.csv')

Теперь парсим историю макроэкономических показателей с сайта rbc.ru. Я не смог быстро разобраться, как парсить всплывающие окна, поэтому просто сохранил коды всплывающих окон в файлы (в этих окнах как раз история).

In [ ]:
def macro(num):
    file = open(str(num), 'r')
    page = file.read()
    file.close()
    page = BeautifulSoup(page, 'lxml')
    features = page.findAll('span',attrs={'class':'macro__indicators__chart__inner'})
    tabl = {'month':int(re.sub(',','.',re.sub('\n| ','',html_stripper(features[6])))[3:5]),
            'year':int(re.sub(',','.',re.sub('\n| ','',html_stripper(features[6])))[6:]),
            'time':re.sub(',','.',re.sub('\n| ','',html_stripper(features[0]))),
            'value':re.sub(',','.',re.sub('\n| ','',html_stripper(features[1]))),
            'change':re.sub(',','.',re.sub('\n| ','',html_stripper(features[2]))),
            'MOM':re.sub(',','.',re.sub('\n| ','',html_stripper(features[3]))),
            'QOQ':re.sub(',','.',re.sub('\n| ','',html_stripper(features[4]))),
            'YOY':re.sub(',','.',re.sub('\n| ','',html_stripper(features[5])))}
    base = pd.DataFrame(tabl,index=[0])
    for i in range(1,int(len(features)/7)):
        tabl = {'month':int(re.sub(',','.',re.sub('\n| ','',html_stripper(features[6+7*i])))[3:5]),
                'year':int(re.sub(',','.',re.sub('\n| ','',html_stripper(features[6+7*i])))[6:]),
                'time':re.sub(',','.',re.sub('\n| ','',html_stripper(features[7*i]))),
                'value':re.sub(',','.',re.sub('\n| ','',html_stripper(features[1+7*i]))),
                'change':re.sub(',','.',re.sub('\n| ','',html_stripper(features[2+7*i]))),
                'MOM':re.sub(',','.',re.sub('\n| ','',html_stripper(features[3+7*i]))),
                'QOQ':re.sub(',','.',re.sub('\n| ','',html_stripper(features[4+7*i]))),
                'YOY':re.sub(',','.',re.sub('\n| ','',html_stripper(features[5+7*i])))}
        base = base.append(pd.DataFrame(tabl,index=[i]))
        
    base = base[(base['year']>2008) | ((base['year']==2008) & (base['month']>=3))]
    base.drop_duplicates(inplace=True,keep='first',subset='time')
    if num not in [7,8]:
        del base['MOM']
        del base['QOQ']
        del base['YOY']
        del base['change']
    else:
        base['value'] = base['MOM']
        del base['MOM']
        del base['QOQ']
        del base['YOY']
        del base['change']
    base.to_csv(str(num)+'.csv',index=False)

Собсвтенно, обрабатываем эти файлы

In [ ]:
for i in range(24):
    macro(i)

Добавляем столбцы с месяцем и годом

In [ ]:
M = [3,2,1]+[12,11,10,9,8,7,6,5,4,3,2,1]*8+[12,11,10,9,8,7,6,5,4,3]
Y = [2017]*3+[2016]*12+[2015]*12+[2014]*12+[2013]*12+[2012]*12+[2011]*12+[2010]*12+[2009]*12+[2008]*10
base = pd.read_csv('0.csv')
base['month'] = M
base['year'] = Y
del base['time']
for i in range(1,20):
    base2 = pd.read_csv(str(i)+'.csv')
    base['value'+str(i)] = base2['value']
base.to_csv('macro.csv',index=False)

Где-то нашел историю курса рубля к доллару

In [ ]:
kurs = pd.read_csv('usd_quotes.csv',sep=';')[:2300]
kurs['day'] = 0
kurs['month'] = 0
kurs['year'] = 0
for i in range(2300):
    kurs['year'][i], kurs['month'][i], kurs['day'][i], = kurs['updated'][i].split('-')
        

Добавляем столбец курса рубля к макрокономическим показателям

In [ ]:
tend = kurs.drop_duplicates(keep='last',subset=['month','year'])
tend = tend.reset_index(drop=True)
tend = tend[:-1]
base['kurs'] = tend['price']
base.to_csv('macro.csv',index=False)